# 第5章: 係り受け解析
日本語Wikipediaの「人工知能」に関する記事からテキスト部分を抜き出したファイルがai.ja.zipに収録されている． この文章をCaboChaやKNP等のツールを利用して係り受け解析を行い，その結果をai.ja.txt.parsedというファイルに保存せよ．このファイルを読み込み，以下の問に対応するプログラムを実装せよ

# 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ

In [1]:
import re

class Morph:
    def __init__(self, morph):
        morph = re.split("[\t,]", morph)
        if len(morph) >= 8:
            self.surface = morph[0]
            self.base = morph[7]
            self.pos = morph[1]
            self.pos1 = morph[2]
            
with open("../output/ai.ja.txt.parsed", mode="r") as f_parsed:
    sentences = []
    sentence = []
    for line in f_parsed:
        if line.startswith("*"):
            continue
        elif line.startswith("EOS"):
            if len(sentence) >= 1: 
                sentences.append(sentence)
            sentence = []
        else:
            sentence.append(Morph(line.rstrip()))
        
for morph in sentences[1]:
    print(morph.surface, morph.base, morph.pos, morph.pos1)
            

人工 人工 名詞 一般
知能 知能 名詞 一般
（ （ 記号 括弧開
じん じん 名詞 一般
こうち こうち 名詞 一般
のう のう 助詞 終助詞
、 、 記号 読点
、 、 記号 読点
AI * 名詞 一般
〈 〈 記号 括弧開
エーアイ * 名詞 固有名詞
〉 〉 記号 括弧閉
） ） 記号 括弧閉
と と 助詞 格助詞
は は 助詞 係助詞
、 、 記号 読点
「 「 記号 括弧開
『 『 記号 括弧開
計算 計算 名詞 サ変接続
（ （ 記号 括弧開
） ） 記号 括弧閉
』 』 記号 括弧閉
という という 助詞 格助詞
概念 概念 名詞 一般
と と 助詞 並立助詞
『 『 記号 括弧開
コンピュータ コンピュータ 名詞 一般
（ （ 記号 括弧開
） ） 記号 括弧閉
』 』 記号 括弧閉
という という 助詞 格助詞
道具 道具 名詞 一般
を を 助詞 格助詞
用い 用いる 動詞 自立
て て 助詞 接続助詞
『 『 記号 括弧開
知能 知能 名詞 一般
』 』 記号 括弧閉
を を 助詞 格助詞
研究 研究 名詞 サ変接続
する する 動詞 自立
計算 計算 名詞 サ変接続
機 機 名詞 接尾
科学 科学 名詞 一般
（ （ 記号 括弧開
） ） 記号 括弧閉
の の 助詞 連体化
一 一 名詞 数
分野 分野 名詞 一般
」 」 記号 括弧閉
を を 助詞 格助詞
指す 指す 動詞 自立
語 語 名詞 一般
。 。 記号 句点
「 「 記号 括弧開
言語 言語 名詞 一般
の の 助詞 連体化
理解 理解 名詞 サ変接続
や や 助詞 並立助詞
推論 推論 名詞 サ変接続
、 、 記号 読点
問題 問題 名詞 ナイ形容詞語幹
解決 解決 名詞 サ変接続
など など 助詞 副助詞
の の 助詞 連体化
知的 知的 名詞 一般
行動 行動 名詞 サ変接続
を を 助詞 格助詞
人間 人間 名詞 一般
に に 助詞 格助詞
代わっ 代わる 動詞 自立
て て 助詞 接続助詞
コンピューター コンピューター 名詞 一般
に に 助詞 格助詞
行わ 行う 動詞 自立
せる せる 動詞 接尾
技術 技術 名詞 一般
」 」 記号 括弧閉
、 、 記号 読点
または または 接続詞 *
、 、 記号 読点
「 「 記号 括弧開
計算 計算 名詞 サ変接続
機

# 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．



In [2]:
class Chunk:
    def __init__(self, dst):
        self.dst = dst
        self.srcs = []
        self.morphs = []
        
    def add_src(self, src: int):
        self.srcs.append(src)
        
    def add_morph(self, morph: Morph):
        self.morphs.append(morph)
        
with open("../output/ai.ja.txt.parsed", mode="r") as f_parsed:
    sentences_chunk = []
    sentence_chunk = []
    chunk = None
    for line in f_parsed:
        if line.startswith("EOS"):
            if not chunk is  None:
                sentence_chunk.append(chunk)
                chunk = None
            if len(sentence_chunk) != 0:
                for i, c in enumerate(sentence_chunk):
                    if c.dst != -1:
                        sentence_chunk[c.dst].add_src(i)
                sentences_chunk.append(sentence_chunk)
            sentence_chunk = []
        else:
            if line.startswith("*"):
                if not chunk is  None:
                    sentence_chunk.append(chunk)
                chunk = Chunk(int(line.split()[2][:-1]))
            else:
                chunk.add_morph(Morph(line.rstrip()))
        
for i, chunk in enumerate(sentences_chunk[1]):
    print(f"idx: {i}, dst: {chunk.dst}, from: {chunk.srcs}")
    print("\t", "".join([morph.surface for morph in chunk.morphs]))

idx: 0, dst: 17, from: []
	 人工知能
idx: 1, dst: 17, from: []
	 （じんこうちのう、、
idx: 2, dst: 3, from: []
	 AI
idx: 3, dst: 17, from: [2]
	 〈エーアイ〉）とは、
idx: 4, dst: 5, from: []
	 「『計算
idx: 5, dst: 9, from: [4]
	 （）』という
idx: 6, dst: 9, from: []
	 概念と
idx: 7, dst: 8, from: []
	 『コンピュータ
idx: 8, dst: 9, from: [7]
	 （）』という
idx: 9, dst: 10, from: [5, 6, 8]
	 道具を
idx: 10, dst: 12, from: [9]
	 用いて
idx: 11, dst: 12, from: []
	 『知能』を
idx: 12, dst: 13, from: [10, 11]
	 研究する
idx: 13, dst: 14, from: [12]
	 計算機科学
idx: 14, dst: 15, from: [13]
	 （）の
idx: 15, dst: 16, from: [14]
	 一分野」を
idx: 16, dst: 17, from: [15]
	 指す
idx: 17, dst: 34, from: [0, 1, 3, 16]
	 語。
idx: 18, dst: 20, from: []
	 「言語の
idx: 19, dst: 20, from: []
	 理解や
idx: 20, dst: 21, from: [18, 19]
	 推論、
idx: 21, dst: 22, from: [20]
	 問題解決などの
idx: 22, dst: 24, from: [21]
	 知的行動を
idx: 23, dst: 24, from: []
	 人間に
idx: 24, dst: 26, from: [22, 23]
	 代わって
idx: 25, dst: 26, from: []
	 コンピューターに
idx: 26, dst: 27, from: [24, 25]
	 行わせる
idx: 27, dst: 34, from:

# 42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [3]:
chunks = sentences_chunk[1]
for chunk in chunks:
    src_s = "".join([morph.surface if not morph.pos == "記号" else "" for morph in chunk.morphs])
    if chunk.dst == -1:
        dst_s = "<none>"
    else:
        dst_s = "".join([morph.surface if not morph.pos == "記号" else "" for morph in chunks[chunk.dst].morphs])
    print(src_s, dst_s, sep="\t")
    

人工知能	語
じんこうちのう	語
AI	エーアイとは
エーアイとは	語
計算	という
という	道具を
概念と	道具を
コンピュータ	という
という	道具を
道具を	用いて
用いて	研究する
知能を	研究する
研究する	計算機科学
計算機科学	の
の	一分野を
一分野を	指す
指す	語
語	研究分野とも
言語の	推論
理解や	推論
推論	問題解決などの
問題解決などの	知的行動を
知的行動を	代わって
人間に	代わって
代わって	行わせる
コンピューターに	行わせる
行わせる	技術または
技術または	研究分野とも
計算機	コンピュータによる
コンピュータによる	情報処理システムの
知的な	情報処理システムの
情報処理システムの	実現に関する
設計や	実現に関する
実現に関する	研究分野とも
研究分野とも	される
される	<none>


# 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [4]:
for chunk in chunks:
    if chunk.dst == -1:
        continue
    if "名詞" in [morph.pos for morph in chunk.morphs]:
        if "動詞" in [morph.pos for morph in chunks[chunk.dst].morphs]:
            src_s = "".join([morph.surface if not morph.pos == "記号" else "" for morph in chunk.morphs])
            dst_s = "".join([morph.surface if not morph.pos == "記号" else "" for morph in chunks[chunk.dst].morphs])
            print(src_s, dst_s, sep="\t")

道具を	用いて
知能を	研究する
一分野を	指す
知的行動を	代わって
人間に	代わって
コンピューターに	行わせる
研究分野とも	される


# 44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，Graphviz等を用いると

# 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

- 動詞を含む文節において，最左の動詞の基本形を述語とする
- 述語に係る助詞を格とする
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．

```
作り出す	で は を
```
このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

- コーパス中で頻出する述語と格パターンの組み合わせ
- 「行う」「なる」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）


In [5]:
with open("../output/corpus_verb_pp.txt", mode="w") as f:
    for chunks in sentences_chunk:
        i = -1
        for chunk in chunks:
            i += 1
            if len(chunk.srcs)== 0:
                continue
            src_chunks = [chunks[i] for i in chunk.srcs]
            if "動詞" in [morph.pos for morph in chunk.morphs]:
                verb = [morph.base for morph in chunk.morphs if morph.pos == "動詞"][0]
                pp = []
                for chunk in src_chunks:
                    for morph in chunk.morphs[::-1]:
                        if morph.pos == "助詞":
                            pp.append(morph.base)
                            break
                pp = sorted(set(pp))
                if len(pp) == 0: 
                    continue
                f.write(verb + "\t" + " ".join(pp) + "\n")

In [6]:
!cat ../output/corpus_verb_pp.txt | sort | uniq -c | sort -n -r | head

  51 する	を
  20 する	が
  19 する	と
  17 する	に
  13 する	は を
  11 よる	に
  10 する	に を
   9 する	で を
   8 する	と は
   8 する	が に


In [7]:
!sort ../output/corpus_verb_pp.txt | uniq -c | sort -n -r | grep 行う | head

   8 行う	を
   1 行う	まで を
   1 行う	から
   1 行う	に により を
   1 行う	に まで を
   1 行う	は を をめぐって
   1 行う	が て に は
   1 行う	で に を
   1 行う	て に を
   1 行う	が で は


In [8]:
!sort ../output/corpus_verb_pp.txt | uniq -c | sort -n -r | grep なる | head 

   4 なる	が と
   3 なる	に は
   2 なる	に
   2 なる	と
   1 無くなる	は
   1 異なる	が で
   1 異なる	も
   1 なる	から が て と は
   1 なる	から で と
   1 なる	が にとって は


In [9]:
!sort ../output/corpus_verb_pp.txt | uniq -c | sort -n -r | grep 与える | head 

   2 与える	が に
   1 与える	に は を


# 46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

- 項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
- 述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．
```
作り出す	で は を	会議で ジョンマッカーシーは 用語を
```

In [10]:
with open("../output/corpus_verb_pp_term.txt", mode="w") as f:
    for chunks in sentences_chunk:
        for chunk in chunks:
            if len(chunk.srcs)== 0:
                continue
            src_chunks = [chunks[i] for i in chunk.srcs]
            if "動詞" in [morph.pos for morph in chunk.morphs]:
                verb = [morph.base for morph in chunk.morphs if morph.pos == "動詞"][0]
                pp_term = []
                for chunk in src_chunks:
                    for morph in chunk.morphs[::-1]:
                        if morph.pos == "助詞":
                            term = "".join([morph.surface for morph in chunk.morphs if morph.pos != "記号"])
                            pp_term.append((morph.base, term))
                            break
                pp_term = sorted(pp_term, key=lambda x: x[0])
                if len(pp_term) == 0: 
                    continue
                pp, term = map(list, zip(*pp_term))
                f.write(verb + "\t" + " ".join(pp) + "\t" + " ".join(term) + "\n")
                
                


# 47. 機能動詞構文のマイニング
動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

- 「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
- 述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
- 述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）
例えば「また、自らの経験を元に学習を行う強化学習という手法もある。」という文から，以下の出力が得られるはずである．
```
学習を行う	に を	元に 経験を
```

In [11]:
with open("../output/corpus_verb_pp_term_sahen.txt", mode="w") as f:
    for chunks in sentences_chunk:
        for chunk in chunks:
            if len(chunk.srcs)== 0:
                continue
            src_chunks = [chunks[i] for i in chunk.srcs]
            if "動詞" in [morph.pos for morph in chunk.morphs]:
                    verb = [morph.base for morph in chunk.morphs if morph.pos == "動詞"][0]
                    pp_term = []
                    sahen_term = None
                    for chunk in src_chunks[::-1]:
                        for i, morph in enumerate(chunk.morphs[::-1]):
                            if morph.pos == "助詞":
                                term = "".join([morph.surface for morph in chunk.morphs if morph.pos != "記号"])
                                pp_term.append((morph.base, term))
                                if morph.base == "を" and chunk.morphs[-(i+2)].pos1 == "サ変接続" and sahen_term is None:
                                    sahen_term = chunk.morphs[-(i+2)].surface + "を"
                                    pp_term.pop()
                                break
                    if sahen_term is None:
                        continue
                    pp_term = sorted(pp_term, key=lambda x: x[0])
                    if len(pp_term) == 0: 
                        continue
                    pp, term = map(list, zip(*pp_term))
                    f.write(sahen_term + verb + "\t" + " ".join(pp) + "\t" + " ".join(term) + "\n")

In [12]:
!cat ../output/corpus_verb_pp_term_sahen.txt | cut -f 1 | sort | uniq -c | sort -nr | head -n 10

   3 注目を集める
   2 開発をする
   2 運転をする
   2 制御をする
   2 処理を行う
   1 存在を見いだす
   1 進化を見せる
   1 追及を受ける
   1 行動を代わる
   1 禁止を求める


In [13]:
!cat ../output/corpus_verb_pp_term_sahen.txt | cut -f 1,2 | sort | uniq -c | sort -nr | head -n 10

   1 存在を見いだす	に
   1 研究を始める	とも は
   1 注目を集める	から は
   1 反乱を起こす	て に対して
   1 進化を見せる	て て において は
   1 追及を受ける	て で と とともに は
   1 改善を果たす	が で に
   1 禁止を求める	が は
   1 研究を続ける	が て
   1 制御を用いる	て も


# 48. 名詞から根へのパスの抽出
文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

- 各文節は（表層形の）形態素列で表現する
- パスの開始文節から終了文節に至るまで，各文節の表現を” -> “で連結する
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．
```
ジョンマッカーシーは -> 作り出した
AIに関する -> 最初の -> 会議で -> 作り出した
最初の -> 会議で -> 作り出した
会議で -> 作り出した
人工知能という -> 用語を -> 作り出した
用語を -> 作り出した
```
KNPを係り受け解析に用いた場合，次のような出力が得られると思われる．
```
ジョンマッカーシーは -> 作り出した
ＡＩに -> 関する -> 会議で -> 作り出した
会議で -> 作り出した
人工知能と -> いう -> 用語を -> 作り出した
用語を -> 作り出した
```

In [14]:
with open("../output/path_noun_to_root.txt", mode="w") as f:
    dict_pathes = dict()
    lines = []
    for j, chunks in enumerate(sentences_chunk):
        pathes = []
        path = []
        for i, chunk in enumerate(chunks):
            if "名詞" in [morph.pos for morph in chunk.morphs]:
                line = ["".join([morph.surface for morph in chunk.morphs if morph.pos != "記号"])]
                path = [i]
                now_chunk = chunk
                while now_chunk.dst != -1:
                    if "句点" in [morph.pos1 for morph in now_chunk.morphs]:
                        break
                    path.append(now_chunk.dst)
                    now_chunk = chunks[now_chunk.dst]
                    line.append("".join([morph.surface for morph in now_chunk.morphs if morph.pos != "記号"]))
                if len(line) > 1:
                    f.write(" -> ".join(line) + "\n")
                    lines.append(line)
                    pathes.append(path)
        dict_pathes[j] = pathes

# 49. 名詞間の係り受けパスの抽出
文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がiとj（i<j）のとき，係り受けパスは以下の仕様を満たすものとする．

- 問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を” -> “で連結して表現する
- 文節iとjに含まれる名詞句はそれぞれ，XとYに置換する
また，係り受けパスの形状は，以下の2通りが考えられる．

- 文節iから構文木の根に至る経路上に文節jが存在する場合: 文節iから文節jのパスを表示
- 上記以外で，文節iと文節jから構文木の根に至る経路上で共通の文節kで交わる場合: 文節iから文節kに至る直前のパスと文節jから文節kに至る直前までのパス，文節kの内容を” | “で連結して表示
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．
```
Xは | Yに関する -> 最初の -> 会議で | 作り出した
Xは | Yの -> 会議で | 作り出した
Xは | Yで | 作り出した
Xは | Yという -> 用語を | 作り出した
Xは | Yを | 作り出した
Xに関する -> Yの
Xに関する -> 最初の -> Yで
Xに関する -> 最初の -> 会議で | Yという -> 用語を | 作り出した
Xに関する -> 最初の -> 会議で | Yを | 作り出した
Xの -> Yで
Xの -> 会議で | Yという -> 用語を | 作り出した
Xの -> 会議で | Yを | 作り出した
Xで | Yという -> 用語を | 作り出した
Xで | Yを | 作り出した
Xという -> Yを
```
KNPを係り受け解析に用いた場合，次のような出力が得られると思われる．
```
Xは | Yに -> 関する -> 会議で | 作り出した。
Xは | Yで | 作り出した。
Xは | Yと -> いう -> 用語を | 作り出した。
Xは | Yを | 作り出した。
Xに -> 関する -> Yで
Xに -> 関する -> 会議で | Yと -> いう -> 用語を | 作り出した。
Xに -> 関する -> 会議で | Yを | 作り出した。
Xで | Yと -> いう -> 用語を | 作り出した。
Xで | Yを | 作り出した。
Xと -> いう -> Yを
```

In [15]:
for k, v in dict_pathes.items():
    print(k, ":", v)
    print()

0 : []

1 : [[0, 17], [1, 17], [2, 3, 17], [3, 17], [4, 5, 9, 10, 12, 13, 14, 15, 16, 17], [6, 9, 10, 12, 13, 14, 15, 16, 17], [7, 8, 9, 10, 12, 13, 14, 15, 16, 17], [9, 10, 12, 13, 14, 15, 16, 17], [11, 12, 13, 14, 15, 16, 17], [12, 13, 14, 15, 16, 17], [13, 14, 15, 16, 17], [15, 16, 17], [18, 20, 21, 22, 24, 26, 27, 34, 35], [19, 20, 21, 22, 24, 26, 27, 34, 35], [20, 21, 22, 24, 26, 27, 34, 35], [21, 22, 24, 26, 27, 34, 35], [22, 24, 26, 27, 34, 35], [23, 24, 26, 27, 34, 35], [25, 26, 27, 34, 35], [27, 34, 35], [28, 29, 31, 33, 34, 35], [29, 31, 33, 34, 35], [30, 31, 33, 34, 35], [31, 33, 34, 35], [32, 33, 34, 35], [33, 34, 35], [34, 35]]

2 : [[0, 1, 5], [1, 5], [2, 3, 5], [3, 5], [4, 5]]

3 : [[0, 1, 3, 5], [1, 3, 5], [2, 3, 5], [3, 5], [4, 5], [6, 19], [7, 8, 9, 10, 11, 12, 18, 19], [8, 9, 10, 11, 12, 18, 19], [9, 10, 11, 12, 18, 19], [10, 11, 12, 18, 19], [11, 12, 18, 19], [12, 18, 19], [13, 14, 17, 18, 19], [14, 17, 18, 19], [15, 16, 17, 18, 19], [16, 17, 18, 19], [17, 18, 19], 

In [40]:
import copy

with open("../output/path_nounX_to_nounY.txt", mode="w") as f:
    for i, pathes in dict_pathes.items():
        for j, path_X in enumerate(pathes[:-1]):
            for path_Y in pathes[j+1:]:
                s_X = set(path_X)
                s_Y = set(path_Y)
                if s_X >= s_Y:
                    idx = sorted(list(s_X - s_Y) + [path_Y[0]])
                    chunks = copy.deepcopy(sentences_chunk[i])
                    chunks = [chunks[k] for k in range(len(chunks)) if k in idx]
                    if not "名詞" in chunks[0].morphs and "名詞" in chunks[0].morphs:
                        continue
                    for n, morph in enumerate(chunks[0].morphs):
                        if morph.pos == "記号":
                            continue
                        if morph.pos == "名詞" or morph.pos == "接頭詞":
                            chunks[0].morphs[n].surface = "X"
                        else:
                            break
                    for n, morph in enumerate(chunks[-1].morphs):
                        if morph.pos == "記号":
                            continue
                        if morph.pos == "名詞" or morph.pos == "接頭詞":
                            chunks[-1].morphs[n].surface = "Y"
                        else:
                            break
                    s = " -> ".join(["".join([morph.surface for morph in chunk.morphs if morph.pos != "記号"]) for chunk in chunks])
                    s = re.sub("X+", "X", s)
                    s = re.sub("Y+", "Y", s)
                    print(s, file=f)
                elif len(s_X - s_Y) == 1:
                    px = copy.deepcopy(path_X)
                    py = copy.deepcopy(path_Y)
                    px.pop()
                    py.pop()
                    chunks = copy.deepcopy(sentences_chunk[i])
                    chunks_X = [chunks[k] for k in range(len(chunks)) if k in px]
                    chunks_Y = [chunks[k] for k in range(len(chunks)) if k in py]
                    last_chunk = chunks[path_X[-1]]
                    for n, morph in enumerate(chunks_X[0].morphs):
                        if morph.pos == "記号":
                            continue
                        if morph.pos == "名詞" or morph.pos == "接頭詞":
                            chunks_X[0].morphs[n].surface = "X"
                        else:
                            break
                    for n, morph in enumerate(chunks_Y[0].morphs):
                        if morph.pos == "記号":
                            continue
                        if morph.pos == "名詞" or morph.pos == "接頭詞":
                            chunks_Y[0].morphs[n].surface = "Y"
                        else:
                            break
                    s_x = " -> ".join(["".join([morph.surface for morph in chunk.morphs if morph.pos != "記号"]) for chunk in chunks_X])
                    s_y = " -> ".join(["".join([morph.surface for morph in chunk.morphs if morph.pos != "記号"]) for chunk in chunks_Y])
                    s_l = "".join([morph.surface for morph in last_chunk.morphs if morph.pos != "記号"])
                    s = s_x + " | " + s_y + " | " + s_l
                    s = re.sub("X+", "X", s)
                    s = re.sub("Y+", "Y", s)
                    print(s, file=f)